<a href="https://colab.research.google.com/github/Garrafao/WUGs/blob/main/scripts/misc/wic2wug.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [170]:
!wget https://pilehvar.github.io/wic/package/WiC_dataset.zip

--2023-05-26 12:23:34--  https://pilehvar.github.io/wic/package/WiC_dataset.zip
Resolving pilehvar.github.io (pilehvar.github.io)... 185.199.108.153, 185.199.110.153, 185.199.109.153, ...
Connecting to pilehvar.github.io (pilehvar.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 275984 (270K) [application/zip]
Saving to: ‘WiC_dataset.zip.2’

WiC_dataset.zip.2   100%[===================>] 269.52K  --.-KB/s    in 0.04s   

2023-05-26 12:23:35 (6.80 MB/s) - ‘WiC_dataset.zip.2’ saved [275984/275984]



In [171]:
import pandas as pd
import numpy as np
import os
import io
import spacy
nlp = spacy.load("en_core_web_sm")

In [172]:
os.makedirs('WiC_data', exist_ok=True)

In [173]:
path = 'WiC_data'

In [174]:
dev = os.makedirs(path + '/dev', exist_ok=True )
train = os.makedirs(path + '/train', exist_ok=True )
test = os.makedirs(path+ '/test', exist_ok=True )


In [175]:
!pip install spacy  

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [176]:
fold = 'WiC_dataset'

In [177]:
from zipfile import ZipFile
with ZipFile('WiC_dataset.zip', 'r') as WiC:
   WiC.extractall(fold)

In [178]:
ds = os.listdir(fold)

In [179]:
ds.remove('README.txt')

In [180]:
pathss = []
for i in ds:
  pathss.append(fold +'/' + i + '/' + i+'.data.txt')


In [181]:
path_jud = []
for i in ds:
  path_jud.append(fold + '/' + i + '/' + i+'.gold.txt')

In [182]:
df_all = pd.DataFrame()
for i in pathss:
  df_3 = pd.read_csv(i, sep = '\t', names=["lemma", "pos", "indice", "sent1", "sent2"], encoding = 'UTF-8')
  df_3['dataset'] = i.split('/')[1]
  df_all = pd.concat([df_all, df_3])

In [183]:
df_all_labels = pd.DataFrame()
for i in path_jud:
  tmp = pd.read_csv(i, sep = '\t', names = ['judgment'], encoding = 'UTF-8')
  tmp['dataset'] = i.split('/')[1]
  df_all_labels = pd.concat([df_all_labels, tmp])

In [184]:
x= df_all['indice'].tolist()


In [185]:
def create_ident(sent1, ind, lemma):
  x1 = str(ind).split('-')[0]
  id1 = lemma+'_' + x1 +'_' + sent1
  return id1


In [186]:
def create_ident2(sent2, ind, lemma):
  x2 = str(ind).split('-')[1]
  id2 = lemma+'_' + x2 +'_' + sent2 
  return id2

In [187]:
df_ = df_all

In [188]:
df_['id1'] = df_.apply(lambda x: create_ident(x.sent1, x.indice, x.lemma), axis=1)

In [189]:
df_['id2'] = df_.apply(lambda x: create_ident2(x.sent2, x.indice, x.lemma), axis=1)

In [190]:
df_judgments = df_[['id1', 'id2', 'lemma', 'dataset']]
df_judgments = df_judgments.rename(columns = {'id1':'identifier1', 'id2': 'identifier2'})

In [191]:
df_judgments['annotator'] = 'gold'
df_judgments['comment'] = ' '
df_judgments['judgment'] = df_all_labels["judgment"].values.tolist()

In [192]:
df_judgments = df_judgments[["identifier1", "identifier2", "annotator", "judgment", "comment", "lemma", 'dataset']]

In [193]:
df_dash = df_[['lemma', 'sent1', 'sent2', 'dataset', 'id1', 'id2']]

In [194]:
df1 = df_dash[['lemma', 'sent1', 'dataset', 'id1']]
df1.columns =  ['word', 'sent', 'dataset', 'id']

In [195]:
df2 =  df_dash[['lemma', 'sent2', 'dataset', 'id2']]
df2.columns =  ['word', 'sent', 'dataset', 'id']

In [196]:
df_final = pd.concat([df1, df2])
df_final= df_final.sort_values(by = ['word'], ascending = True) 

In [197]:
dftodict = pd.unique(df_final[['sent', 'id']].values.ravel())

In [198]:
result_df = df_final.drop_duplicates()

In [199]:
sent_to_id = pd.Series(result_df.id.values,index=result_df.sent).to_dict()

In [200]:
ind1 = []
ind2 = []
for i in x:
    ind1.append(i.split('-')[0])
    ind2.append(i.split('-')[1])

In [201]:
df_all.loc[:,'ind1'] = ind1
df_all.loc[:,'ind2'] = ind2


In [202]:
df1 = df_all[['lemma', 'pos', 'sent1', 'ind1', 'dataset']]
df1.columns =  ['lemma','pos', 'context_tokenized','ind', 'dataset']


In [203]:
df2 = df_all[['lemma', 'pos', 'sent2', 'ind2', 'dataset']]
df2.columns =  ['lemma','pos', 'context_tokenized','ind', 'dataset']

In [204]:
df_final_uses = pd.concat([df1, df2])
df_final_uses = df_final_uses.sort_values(by = ['lemma'], ascending = True)


In [205]:
df_final_uses = df_final_uses.reset_index(drop =True)

In [206]:
df_final_uses = df_final_uses.drop_duplicates()

In [207]:
def get_indice(sent, indice):
  tag = ''
  indice = int(indice)
  tok = sent.split(" ")
  #print(tok)
  for i in range(len(tok)):
    if i == indice:
      tag = tok[i]
  return str(sent.find(tag))+":"+str(sent.find(tag)+len(tag))

In [208]:
df_final_uses["indexes_target_token"] = df_final_uses.apply(lambda x: get_indice(x.context_tokenized, x.ind), axis=1)


In [209]:
def get_indices_of_sent(sentence):
    return "0:"+str(len(sentence))

In [210]:
df_final_uses["indexes_target_sentence"] = df_final_uses["context_tokenized"].apply(get_indices_of_sent)

In [211]:
def get_len_tok(sentence):
  return "0:"+str(len(sentence.split(" ")))

In [212]:
df_final_uses['indexes_target_sentence_tokenized'] = df_final_uses["context_tokenized"].apply(get_len_tok)


In [213]:
lem = ''
def lemmatizr(sent):
  doc = nlp(sent)
  for i in doc:
    lem += i.lemma_
    return lem

In [214]:
df_final_uses['context_lemmatized'] = df_final_uses["context_tokenized"].apply(lambda row: " ".join([w.lemma_ for w in nlp(row)]))

In [215]:
df_final_uses["indice"] = df_final_uses.groupby("lemma").cumcount()

In [216]:
df_final_uses['context_pos'] = df_final_uses["context_tokenized"].apply(lambda row: " ".join([w.pos_ for w in nlp(row)]))

In [217]:
df_final_uses['date'] = " "
df_final_uses['grouping'] = 1
df_final_uses['description'] = " "
df_final_uses['context'] = df_final_uses['context_tokenized']
#df_final_dev_uses['identifier'] = ""
df_final_uses['indexes_target_token_tokenized'] = df_final_uses['ind']


In [218]:
def get_identifier(sent, dictionar):
  for k, v in sent_to_id.items():
    for i in v:
      if sent == k:
        return v

In [219]:
sents = df_final_uses.context.tolist()

In [220]:
aggre = []
for i in sents:
  aggre.append(get_identifier(i, sent_to_id))

In [221]:
df_final_uses['identifier'] = aggre

In [222]:
final_dfs = df_final_uses[['lemma', 'pos', 'date', 'grouping', 'identifier', 'description', 'context', 'indexes_target_token', 'indexes_target_sentence', 'context_tokenized', 'indexes_target_token_tokenized', 'indexes_target_sentence_tokenized', 'context_lemmatized', 'context_pos', 'dataset']]


In [223]:
final_df = final_dfs.reset_index(drop= True)

In [224]:
!git clone https://github.com/Garrafao/WUGs.git

fatal: destination path 'WUGs' already exists and is not an empty directory.


In [225]:
os.makedirs('/content/WUGs/scripts/misc/data/dwug_en/data/*/', exist_ok=True )

In [226]:
import csv
final_df.to_csv('/content/WUGs/scripts/misc/data/dwug_en/data/*/uses.csv', sep='\t', encoding='utf-8', quoting = csv.QUOTE_NONE, quotechar='')


In [ ]:
%run /content/WUGs/scripts/misc/use2normalize.py /content/WUGs/scripts/misc/data/dwug_en/data/*/uses.csv dwug_en output

In [228]:
final_uses = pd.read_csv('/content/output', delimiter = '\t',quoting =3)


In [229]:
final_uses['context_tokenized'] = final_df['context_tokenized']
final_uses['pos'] = final_df['pos']
final_uses['dataset'] = final_df['dataset']
final_uses['grouping'] = final_df['grouping']
final_uses['lemma'] = final_df['lemma']
final_uses['indexes_target_token_tokenized'] = final_df['indexes_target_token_tokenized']
final_uses['indexes_target_sentence_tokenized'] = final_df['indexes_target_sentence_tokenized']
final_uses['date'] = " "
final_uses['description'] = " "
final_uses['context_lemmatized'] = final_df['context_lemmatized']
final_uses['context_pos'] = final_df['context_pos']


In [230]:
final_uses = final_uses[['lemma', 'pos', 'date', 'grouping', 'identifier', 'description', 'context', 'indexes_target_token', 'indexes_target_sentence', 'context_tokenized', 'indexes_target_token_tokenized', 'indexes_target_sentence_tokenized', 'context_lemmatized', 'context_pos', 'dataset']]


In [231]:
for i in list(final_uses["dataset"].value_counts().index):
    df_temp = final_uses[final_uses["dataset"]==i]
    if not os.path.exists(i):
        os.mkdir(i)
    df_temp.to_csv(path +'/'+ i +'/uses.csv',index = False, sep='\t', encoding='utf-8', quoting=csv.QUOTE_NONE, quotechar = '')

In [232]:
for i in list(df_judgments["dataset"].value_counts().index):
    df_temp = df_judgments[df_judgments["dataset"]==i]
    if not os.path.exists(i):
        os.mkdir(i)
    df_temp.to_csv(path + '/' + i +'/judgments.csv',index = False, sep='\t', encoding='utf-8', quoting=csv.QUOTE_NONE, quotechar = '')